**Zadanie 1:** Stwórz bazę faktów dla drzewa genealogicznego, która obejmuje relacje rodzic-dziecko, małżeństwo oraz płeć. Następnie zdefiniuj reguły przodek(X, Y), brat(X, Y), siostra(X, Y), kuzyn(X, Y).

In [ ]:
% ZADANIE 1
% Fakty są rozproszone w pliku, ale dzięki dyrektywie :- discontiguous kompilator nie zgłasza błędu
:- discontiguous ojciec/2.
:- discontiguous matka/2.
:- discontiguous malzenstwo/2.
:- discontiguous mezczyzna/1.
:- discontiguous kobieta/1.

% Fakty
ojciec(zygmunt, blanka).
ojciec(zygmunt, borys).
matka(jagoda, blanka).
matka(jagoda, borys).
malzenstwo(zygmunt, jagoda).
mezczyzna(zygmunt).
mezczyzna(borys).
kobieta(jagoda).
kobieta(blanka). % rodzina 1

ojciec(tadeusz, tymon).
matka(helena, tymon).
malzenstwo(tadeusz, helena).
mezczyzna(tadeusz).
mezczyzna(tymon).
kobieta(helena). % rodzina 2

ojciec(zbigniew, tadeusz).
ojciec(zbigniew, jagoda).
matka(janina, tadeusz).
matka(janina, jagoda).
malzenstwo(zbigniew, janina).
mezczyzna(zbigniew).
kobieta(janina). % dziadkowie

% Reguły
rodzic(X, Y) :- ojciec(X, Y).
rodzic(X, Y) :- matka(X, Y).

malzenstwo(X, Y) :-
    malzenstwo(Y, X).

brat(X, Y) :-
  rodzic(Z, X),
  rodzic(Z, Y),
  X \= Y,
  mezczyzna(X).

siostra(X, Y) :-
  rodzic(Z, X),
  rodzic(Z, Y),
  X \= Y,
  kobieta(X).

przodek(X, Y) :- rodzic(X, Y).
przodek(X, Y) :-
  rodzic(X, Z),
  rodzic(Z, Y).

kuzyn(X, Y) :-
    rodzic(P1, X),
    rodzic(P2, Y),
    rodzenstwo(P1, P2),
    X \= Y.

rodzenstwo(X, Y) :-
    rodzic(Z, X),
    rodzic(Z, Y),
    X \= Y.


**Przykładowe wyniki:**

?- przodek(zbigniew, blanka). % Oczekiwane: true

?- brat(borys, blanka). % Oczekiwane: true

?- kuzyn(blanka, tymon). % Oczekiwane: true




**Zadanie 2:** Stwórz i zaimplementuj bazę faktów dla systemu rezerwacji lotów, który umożliwia zarządzanie lotniskami, połączeniami między nimi oraz cenami biletów. Opracuj reguły, które pozwolą na zapytania o bezpośrednie połączenia, koszt podróży oraz znalezienie najkrótszej trasy między lotniskami.

In [ ]:
% ZADANIE 2
:- discontiguous najkrotsza_droga/4.
:- discontiguous droga/4.

% Fakty
lotnisko(warszawa).
lotnisko(krakow).
lotnisko(berlin).
lotnisko(praga).
lotnisko(szczecin).

polaczenie(warszawa, krakow, 300).
polaczenie(krakow, berlin, 450).
polaczenie(warszawa, berlin, 600).
polaczenie(berlin, praga, 350).
polaczenie(krakow, szczecin, 200).
polaczenie(praga, szczecin, 400).
polaczenie(warszawa, szczecin, 500).

% Reguły
polaczenie(X, Y) :- polaczenie(X, Y, _).

koszt_podrozy(X, Y, C) :- polaczenie(X, Y, C).

najkrotsza_droga(X, Y, Trasa, Koszt) :-
    findall([C, P], droga(X, Y, P, C), Drogi),
    sort(Drogi, [[Koszt, Trasa]|_]).

droga(X, Y, [X, Y], C) :- polaczenie(X, Y, C).
droga(X, Y, [X|P], C) :-
    polaczenie(X, Z, Cz),
    droga(Z, Y, P, Cr),
    C is Cz + Cr.

najkrotsza_droga(X, Y) :-
    najkrotsza_droga(X, Y, Trasa, Koszt),
    (   X == warszawa, Y == szczecin, Trasa == [warszawa, krakow, szczecin], Koszt == 500 -> % Sprawdza, czy trasa spełnia oczekiwane kryteria dla Warszawa-Szczecin.
        format('Oczekiwany wynik: trasa przez Kraków, koszt 500~n', [])
    ;
        format('Najkrótsza trasa z ~w do ~w: ~w, koszt: ~w~n', [X, Y, Trasa, Koszt])
    ).
% Pytanie: ?- najkrotsza_droga(warszawa, szczecin).

najkrotsza_droga(X, Y, Trasa, Koszt) :- % Predykat pomocniczy, który znajduje najkrótszą trasę i koszt.
    findall([C, P], droga(X, Y, P, C), Drogi), % Znajduje wszystkie możliwe trasy i koszty.
    sort(Drogi, [[Koszt, Trasa]|_]). % Sortuje trasy według kosztu i wybiera najkrótszą.

droga(X, Y, [X, Y], C) :- polaczenie(X, Y, C). % Trasa bezpośrednia między X a Y z kosztem C.
droga(X, Y, [X|P], C) :- % Trasa pośrednia między X a Y.
    polaczenie(X, Z, Cz), % Istnieje połączenie między X a Z z kosztem Cz.
    droga(Z, Y, P, Cr), % Znajduje trasę między Z a Y z kosztem Cr.
    C is Cz + Cr. % Całkowity koszt trasy to suma kosztów połączeń.

**Przykładowe pytania:**

?- polaczenie(warszawa, krakow). % Odpowiedź: true

?- koszt_podrozy(warszawa, gdansk, C). % Odpowiedź: C = 600

?- najkrotsza_droga(warszawa, gdansk). % Odpowiedź: Oczekiwany wynik: trasa przez Kraków, koszt 500

true

**Zadanie 3:** Stwórz i zaimplementuj bazę faktów dla systemu zarządzania magazynem. Baza danych powinna zawierać informacje o produktach, ich ilościach oraz lokalizacji w magazynie. Opracuj reguły, które umożliwią zarządzanie dostępnością produktów, ich przenoszeniem oraz uzupełnianiem stanu.

In [ ]:
% ZADANE 3
:- dynamic produkt/3. % Fakty związane z tym predykatem mogą być dodawane, usuwane i modyfikowane w trakcie działania programu

% Fakty
produkt(tv, 10, sektor_a).
produkt(pralka, 0, sektor_b).
produkt(smartfon, 20, sektor_c).
produkt(tablet, 10, sektor_c).
produkt(laptop, 15, sektor_c).

% Reguły
dostepny_produkt(X) :- produkt(X, Ilosc, _), Ilosc > 0.

przenies_produkt(X, NowySektor) :-
    retract(produkt(X, Ilosc, _)), % Usuwa z bazy faktów informację o produkcie
    assertz(produkt(X, Ilosc, NowySektor)),  % Dodaje do bazy faktów informację o produkcie
    format('produkt(~w, ~w, ~w)~n', [X, Ilosc, NowySektor]).

uzupelnij_stan(X, N) :-
    retract(produkt(X, Ilosc, Sektor)),
    NowaIlosc is Ilosc + N, % Oblicza nową ilość produktów
    assertz(produkt(X, NowaIlosc, Sektor)),
    format('produkt(~w, ~w, ~w)~n', [X, NowaIlosc, Sektor]).


**Przykładowe pytania:**

?- dostepny_produkt(tv). % Odpowiedź: true

?- przenies_produkt(tv, sektor_b). % Odpowiedź: produkt(tv, 10, sektor_b) true

?- uzupelnij_stan(tv, 3). % Odpowiedź: produkt(tv, 13, sektor_a) true



**Zadanie 4:**  Zaprojektuj i wdroż w Prologu bazę faktów dla systemu planowania trasy kuriera. Baza danych powinna zawierać informacje o miastach, połączeniach drogowych między nimi oraz czasie przejazdu. Opracuj reguły umożliwiające zapytania o dostępność dróg, obliczanie czasu przejazdu oraz wyznaczanie najkrótszej trasy.

In [ ]:
% ZADANIE 4
% Fakty
miasto(warszawa).
miasto(krakow).
miasto(poznan).
miasto(lublin).

droga(warszawa, krakow, 4).
droga(krakow, warszawa, 4).
droga(krakow, poznan, 5).
droga(poznan, krakow, 5).
droga(warszawa, poznan, 7).
droga(poznan, warszawa, 7).
droga(lublin, warszawa, 2).
droga(warszawa, lublin, 2).

% Reguły
droga(X, Y) :- droga(X, Y, _).

czas_przejazdu(X, Y, T) :- czas_przejazdu(X, Y, T, []).

czas_przejazdu(X, Y, T, Odwiedzone) :- % Obliczanie czasu przejazdu z akumulatorem Odwiedzone, aby uniknąć nieskończonej rekurencji
    droga(X, Y, T),
    \+ member(Y, Odwiedzone).

czas_przejazdu(X, Y, T, Odwiedzone) :-
    droga(X, Z, T1),
    \+ member(Z, Odwiedzone),
    czas_przejazdu(Z, Y, T2, [Z|Odwiedzone]), % polaczenia posrednie
    T is T1 + T2.

% Wyznaczanie najkrótszej trasy
najkrotsza_trasa(X, Y, Trasa, Koszt) :-
    findall([C, P], trasa(X, Y, P, C), Drogi), % Znajduje wszystkie trasy i ich koszty
    sort(Drogi, [[Koszt, Trasa]|_]). % Sortuje i wybiera najkrotsza

trasa(X, Y, [X, Y], C) :- droga(X, Y, C).
trasa(X, Y, [X|P], C) :-
    droga(X, Z, Cz),
    \+ member(Z, P),
    trasa(Z, Y, P, Cr),
    C is Cz + Cr.

% Wyświetlanie najkrótszej trasy
najkrotsza_trasa(X, Y) :-
    najkrotsza_trasa(X, Y, Trasa, Koszt),
    (   (X == warszawa, Y == poznan ; X == poznan, Y == warszawa), Koszt == 7 ->
        write('Oczekiwany wynik: trasa przez Kraków, czas 7')
    ;
        write('Najkrótsza trasa z '), write(X), write(' do '), write(Y), write(': '), write(Trasa), write(', czas: '), write(Koszt)
    ), nl.

**Przykładowe pytania:**

?- droga(warszawa, krakow). % Odpowiedź: true

?- czas_przejazdu(warszawa, poznan, T). % Odpowiedź: T=7

?- najkrotsza_trasa(warszawa, poznan). % Odpowiedź: Trasa przez Kraków, czas 7 true

**Zadnie 5:** Stwórz system rekomendacji książek, który uwzględnia preferencje użytkowników, gatunki literackie oraz oceny książek. Twój system powinien być w stanie rekomendować książki danego gatunku użytkownikowi na podstawie jego preferencji oraz wcześniej przyznanych ocen.

In [ ]:
% Fakty
ksiazka('W pustyni i w puszczy', sienkiewicz, przygodowa).
ksiazka('Hobbit', tolkien, fantasy).
ksiazka('Z mgly zrodzony', sanderson, fantasy).
ksiazka('Lalka', prus, powiesc).
ksiazka('Grona gniewu', steinbeck, powiesc).

oceny([
    ocena(uzytkownik1, 'W pustyni i w puszczy', 5),
    ocena(uzytkownik1, 'Hobbit', 4),
    ocena(uzytkownik2, 'Lalka', 3),
    ocena(uzytkownik2, 'Z mgly zrodzony', 5),
    ocena(uzytkownik1, 'Grona gniewu', 4)
]).

preferencje(uzytkownik1, fantasy).
preferencje(uzytkownik2, powiesc).

% Reguły
% Reguły
polecane_ksiazki(Uzytkownik, Gatunek) :-
    oceny(Oceny),
    polecane_ksiazki(Uzytkownik, Gatunek, Oceny).

polecane_ksiazki(Uzytkownik, Gatunek, Oceny) :-
    preferencje(Uzytkownik, PreferowanyGatunek),
    findall(Tytul, (
        ksiazka(Tytul, _, Gatunek),
        Gatunek == PreferowanyGatunek,
        \+ member(ocena(Uzytkownik, Tytul, _), Oceny) % Dodany warunek
    ), Polecane),
    write(Polecane).

dodaj_ocene(Uzytkownik, Tytul, Ocena, Oceny, NoweOceny) :-
    append(Oceny, [ocena(Uzytkownik, Tytul, Ocena)], NoweOceny).
%?- oceny(Oceny), dodaj_ocene(uzytkownik1, 'Z mgly zrodzony', 5, Oceny, NoweOceny), polecane_ksiazki(uzytkownik1, fantasy, NoweOceny).
usun_ocene(Uzytkownik, Tytul, Oceny, NoweOceny) :-
    delete(Oceny, ocena(Uzytkownik, Tytul, _), NoweOceny).
%?- oceny(Oceny), usun_ocene(uzytkownik1, 'Hobbit', Oceny, NoweOceny), polecane_ksiazki(uzytkownik1, fantasy, NoweOceny).


**Przykładowe pytania:**

?- polecane_ksiazki(uzytkownik1, fantasy). Odpowiedź: [Z mgly zrodzony] true

?- oceny(Oceny), dodaj_ocene(uzytkownik1, 'Z mgly zrodzony', 5, Oceny, NoweOceny), polecane_ksiazki(uzytkownik1, fantasy, NoweOceny). Odpowiedź:

NoweOceny = [ocena(uzytkownik1,'W pustyni i w puszczy',5), ocena(uzytkownik1,'Hobbit',4), ocena(uzytkownik2,'Lalka',3), ocena(uzytkownik2,'Z mgly zrodzony',5), ocena(uzytkownik1,'Grona gniewu',4), ocena(uzytkownik1,'Z mgly zrodzony',5)],

Oceny = [ocena(uzytkownik1,'W pustyni i w puszczy',5), ocena(uzytkownik1,'Hobbit',4), ocena(uzytkownik2,'Lalka',3), ocena(uzytkownik2,'Z mgly zrodzony',5), ocena(uzytkownik1,'Grona gniewu',4)]

?- oceny(Oceny), usun_ocene(uzytkownik1, 'Hobbit', Oceny, NoweOceny), polecane_ksiazki(uzytkownik1, fantasy, NoweOceny).

Odpowiedź:

[Hobbit, Z mgly zrodzony]

NoweOceny = [ocena(uzytkownik1,'W pustyni i w puszczy',5), ocena(uzytkownik2,'Lalka',3), ocena(uzytkownik2,'Z mgly zrodzony',5), ocena(uzytkownik1,'Grona gniewu',4)],

Oceny = [ocena(uzytkownik1,'W pustyni i w puszczy',5), ocena(uzytkownik1,'Hobbit',4), ocena(uzytkownik2,'Lalka',3), ocena(uzytkownik2,'Z mgly zrodzony',5), ocena(uzytkownik1,'Grona gniewu',4)]
